In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from tqdm.notebook import tqdm
from medpy.metric.binary import hd, dc
import scipy.ndimage as ndi
import evaluate

In [ ]:
# Cursor parking space


In [ ]:
patient_folders = ["data/testing/testing/" + x + "/" for x in os.listdir("data/testing/testing/")]
# patient_folders = ["data/training/" + x + "/" for x in os.listdir("data/training/")]
patient_files = [[x + y[:-7] for y in os.listdir(x) if "frame" in y and "gt" not in y] for x in patient_folders]
patient_files_flattened = [element for sublist in patient_files for element in sublist]


images = [{'img': x} for x in patient_files_flattened]

In [ ]:
header_dict = {}
device = "cuda"

In [ ]:
# data/training/patient029/patient029_frame01
# patient029_frame01
# SAVING PATIENT patient029_frame01_ES



def get_name(filename):
    number = (filename.split("/")[-1]).split("_")[0][-3:]
    frame = (filename.split("/")[-1]).split("_")[1][-2:]
    res = ""
    if frame == "01":
        res = "patient"+number+"_ED"  
    else:
        res = "patient"+number+"_ES"
        
    return res+".nii.gz"



In [ ]:
from evaluate import load_nii
class LoadNIFTI(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.next_id = 0
        self.headers = {}
        pass

    def __call__(self, sample):
        img_file = sample['img'] + ".nii.gz"
        # img_mask = sample['img'] + "_gt.nii.gz"
                
        image, img_affine, img_header = load_nii(img_file)
        image = np.moveaxis(image, (2), (0))
        
        header_dict[sample['img']]=img_header
        
        # mask, mask_affine, mask_header = load_nii(img_mask)
        # mask = np.moveaxis(mask, (2), (0))
        extra_data = {
            'name': sample['img'], 
            'affine': img_affine, 
            'original': image,
            'scaling': img_header['pixdim'],
            'id': self.next_id
        }
        self.headers[self.next_id] = img_header
        self.next_id += 1
        
        return {'img': image, 'extra_data': extra_data}
    
    def inverse(self, sample):
        img_header = self.headers[sample['extra_data']['id']]
        name = sample['extra_data']['name']
        
        # plt.imshow(sample['img'][0][4])
        # plt.show()
        # plt.imshow(sample['extra_data']['original'][4])
        # plt.show()
        
        reshaped = np.round(np.moveaxis(sample['img'][0].numpy(), (0, 1, 2), (2, 0, 1))).astype(np.uint8)

        img_header['datatype'] = 2
        
        
        evaluate.save_nii("results/validation_masks/"+get_name(name), reshaped, sample['extra_data']['affine'], img_header)
    
       
class ScaleDims(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.transforms = dict()
        pass
    
    def __call__(self, sample):
        scaling = sample['extra_data']['scaling']
        transform = monai.transforms.Zoomd(keys=['img'], mode=['area'], zoom=(scaling[3] / 10, scaling[1] / 1.5, scaling[2] / 1.5), keep_size=False)
        self.transforms[sample['extra_data']['id']] = transform
        return transform(sample)
    
    def inverse(self, sample):
        return self.transforms[sample['extra_data']['id']].inverse(sample)
    
class FindCenter(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.model = monai.networks.nets.Unet(
                spatial_dims=3,
                in_channels=1,
                out_channels=3,
                channels = (8, 16, 32, 64),
                strides=(1, 1, 1),
                num_res_units=2,
            ).to(device)
        
        self.transforms = dict()
        self.model.load_state_dict(torch.load("models/trainedUNet1655808833.8215299_24.pt"))
        self.model.eval
        
    def __call__(self, sample):
        img = sample['img']
        with torch.no_grad():
            out = self.model(torch.unsqueeze(torch.Tensor(img).to(device), dim=0))
            combined = np.sum(out[0].detach().cpu().numpy(), axis=(0, 1))
            cx, cy = ndi.center_of_mass(combined)
            dim_height = out.shape[2]
            self.transforms[sample['extra_data']['id']] = monai.transforms.SpatialCropd(keys=['img'], roi_size=(dim_height, 128, 128), roi_center=(dim_height // 2, int(cx), int(cy)))
            return self.transforms[sample['extra_data']['id']](sample)

    def inverse(self, sample):
        return self.transforms[sample['extra_data']['id']].inverse(sample)


In [ ]:
# Define transforms for loading the dataset

# add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask'])
# flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
# rotate_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])

# compose_transform = monai.transforms.Compose(
#     [
#         LoadNIFTI(),
#         monai.transforms.AddChanneld(keys=['img', 'mask']),
#         monai.transforms.ScaleIntensityd(keys=['img', 'mask'], minv=0.0, maxv=1.0),
#         SplitMask(),
#         monai.transforms.Resized(keys=['img', 'mask'], spatial_size=(-1, 128, 128)),
#         monai.transforms.SpatialPadd(keys=['img', 'mask'], spatial_size=(16, -1, -1)),
#         monai.transforms.SpatialCropd(keys=['img', 'mask'], roi_size=(16, 128, 128), roi_center=(8, 64, 64)),
#         monai.transforms.ScaleIntensityd(keys=['mask'], minv=0.0, maxv=1.0)
#     ]
# )

compose_transform = monai.transforms.Compose(
    [
        LoadNIFTI(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityd(keys=['img'], minv=0.0, maxv=1.0),
        ScaleDims(),
        FindCenter(),
    ]
)
        

In [ ]:
train_dict_list = [x for x in images]
dataset = monai.data.CacheDataset(train_dict_list, transform=compose_transform)


Loading dataset: 100%|██████████| 100/100 [00:09<00:00, 10.98it/s]


In [ ]:
data_loader = monai.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# model = monai.networks.nets.UNETR(in_channels=1, out_channels=3, img_size=(16,128,128), feature_size=32, norm_name='batch').to(device)
model = monai.networks.nets.Unet(
    spatial_dims=3,
    in_channels=1,
    out_channels=3,
    channels = (8, 16, 32, 64),
    strides=(1, 1, 1),
    num_res_units=2,
).to(device)

model.load_state_dict(torch.load('models/trainedUNet1655812377.0684168_136.pt'))

<All keys matched successfully>

In [ ]:
# def flatten(mask):
#     out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
#     return out

# import evaluate

# for d in data_loader:
#     img = d['img']
#     label = d['mask']
#     pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
#     # print(pred.shape)
#     flatt_pred = flatten(pred[0])
#     fixed_label = 3*(label[0][0])
#     print(flatt_pred.shape)
#     print(fixed_label.shape)
    
#     print(evaluate.metrics(flatt_pred, fixed_label, [1,1,1]))
    
#     # evaluate.metrics(label, pred, [1, 1, 1])

    
# #     plt.imshow(d['img'][0][0][4], cmap='gray')
# #     plt.show()
# #     out = torch.clamp(model(d['img'].to(device)), min=0, max=1).detach().cpu().numpy()
# #     o = np.concatenate((out[0][0][4], out[0][1][4], out[0][2][4]), axis=1)
    
# #     # a = np.expand_dims(flatten(d['mask'][0, :, 4]), axis=0)
# #     b = np.expand_dims(flatten(out[0])[4],axis=0)
    
# #     # plt.imshow(a[0])
# #     # plt.show()
# #     plt.imshow(b[0])
# #     plt.show()

#     # print(d['img'].shape)
#     # print(evaluate.metrics(a, b, [1, 1, 1]))
#     # break


In [ ]:
def flatten(mask):
    out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
    return torch.Tensor(out)

for d in data_loader:
    
    img = d['img']
    # print(affine.shape)
    pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
    flatt_pred = flatten(pred[0])
    
    d['img'] = torch.unsqueeze(torch.unsqueeze(flatt_pred, dim=0), dim=0)
    monai.transforms.BatchInverseTransform(compose_transform, data_loader)(d)


In [ ]:
# import evaluate

# evaluate.


In [ ]:
evaluate.compute_metrics_on_files("data/training/patient057/patient057_frame01_gt.nii.gz", "results/validation_masks/patient107_ED.nii.gz")

ValueError: operands could not be broadcast together with shapes (428,512,8) (256,216,9) 

In [ ]:
image1, img_affine, img_header = load_nii("results/masks/patient029_ED.nii.gz")
image2, img_affine, img_header = load_nii("data/training/patient029/patient029_frame01_gt.nii.gz")

print(np.max(image1), np.max(image2))
print(image1.dtype)
evaluate.metrics(image1, image2, (1, 1, 1))
# print(np.max(image1), np.max(image2))


3 3
uint8


[0.9315255076770679,
 8.405,
 0.6579999999999995,
 0.8506687647521637,
 6.188,
 -0.3340000000000005,
 0.8791878172588833,
 10.527,
 -0.6160000000000014]

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import monai
from PIL import Image
import torch
from monai.visualize import blend_images, matshow3d, plot_2d_or_3d_image
from tqdm.notebook import tqdm
from medpy.metric.binary import hd, dc
import scipy.ndimage as ndi
import evaluate

In [ ]:
# Cursor parking space


In [ ]:
patient_folders = ["data/testing/testing/" + x + "/" for x in os.listdir("data/testing/testing/")]
# patient_folders = ["data/training/" + x + "/" for x in os.listdir("data/training/")]
patient_files = [[x + y[:-7] for y in os.listdir(x) if "frame" in y and "gt" not in y] for x in patient_folders]
patient_files_flattened = [element for sublist in patient_files for element in sublist]


images = [{'img': x} for x in patient_files_flattened]

In [ ]:
header_dict = {}
device = "cuda"

In [ ]:
# data/training/patient029/patient029_frame01
# patient029_frame01
# SAVING PATIENT patient029_frame01_ES



def get_name(filename):
    number = (filename.split("/")[-1]).split("_")[0][-3:]
    frame = (filename.split("/")[-1]).split("_")[1][-2:]
    res = ""
    if frame == "01":
        res = "patient"+number+"_ED"  
    else:
        res = "patient"+number+"_ES"
        
    return res+".nii.gz"



In [ ]:
from evaluate import load_nii
class LoadNIFTI(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.next_id = 0
        self.headers = {}
        pass

    def __call__(self, sample):
        img_file = sample['img'] + ".nii.gz"
        # img_mask = sample['img'] + "_gt.nii.gz"
                
        image, img_affine, img_header = load_nii(img_file)
        image = np.moveaxis(image, (2), (0))
        
        header_dict[sample['img']]=img_header
        
        # mask, mask_affine, mask_header = load_nii(img_mask)
        # mask = np.moveaxis(mask, (2), (0))
        extra_data = {
            'name': sample['img'], 
            'affine': img_affine, 
            'original': image,
            'scaling': img_header['pixdim'],
            'id': self.next_id
        }
        self.headers[self.next_id] = img_header
        self.next_id += 1
        
        return {'img': image, 'extra_data': extra_data}
    
    def inverse(self, sample):
        img_header = self.headers[sample['extra_data']['id']]
        name = sample['extra_data']['name']
        
        # plt.imshow(sample['img'][0][4])
        # plt.show()
        # plt.imshow(sample['extra_data']['original'][4])
        # plt.show()
        
        reshaped = np.round(np.moveaxis(sample['img'][0].numpy(), (0, 1, 2), (2, 0, 1))).astype(np.uint8)

        img_header['datatype'] = 2
        
        
        evaluate.save_nii("results/validation_masks/"+get_name(name), reshaped, sample['extra_data']['affine'], img_header)
    
       
class ScaleDims(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.transforms = dict()
        pass
    
    def __call__(self, sample):
        scaling = sample['extra_data']['scaling']
        transform = monai.transforms.Zoomd(keys=['img'], mode=['area'], zoom=(scaling[3] / 10, scaling[1] / 1.5, scaling[2] / 1.5), keep_size=False)
        self.transforms[sample['extra_data']['id']] = transform
        return transform(sample)
    
    def inverse(self, sample):
        return self.transforms[sample['extra_data']['id']].inverse(sample)
    
class FindCenter(monai.transforms.InvertibleTransform):
    def __init__(self, keys=None):
        self.model = monai.networks.nets.Unet(
                spatial_dims=3,
                in_channels=1,
                out_channels=3,
                channels = (8, 16, 32, 64),
                strides=(1, 1, 1),
                num_res_units=2,
            ).to(device)
        
        self.transforms = dict()
        self.model.load_state_dict(torch.load("models/trainedUNet1655808833.8215299_24.pt"))
        self.model.eval
        
    def __call__(self, sample):
        img = sample['img']
        with torch.no_grad():
            out = self.model(torch.unsqueeze(torch.Tensor(img).to(device), dim=0))
            combined = np.sum(out[0].detach().cpu().numpy(), axis=(0, 1))
            cx, cy = ndi.center_of_mass(combined)
            dim_height = out.shape[2]
            self.transforms[sample['extra_data']['id']] = monai.transforms.SpatialCropd(keys=['img'], roi_size=(dim_height, 128, 128), roi_center=(dim_height // 2, int(cx), int(cy)))
            return self.transforms[sample['extra_data']['id']](sample)

    def inverse(self, sample):
        return self.transforms[sample['extra_data']['id']].inverse(sample)


In [ ]:
# Define transforms for loading the dataset

# add_channels_transform = monai.transforms.AddChanneld(keys=['img', 'mask'])
# flip_transform = monai.transforms.RandFlipd(keys=['img', 'mask'], prob=1, spatial_axis=1)
# rotate_transform = monai.transforms.RandRotated(keys=['img', 'mask'], range_x=np.pi/4, prob=1, mode=['bilinear', 'nearest'])

# compose_transform = monai.transforms.Compose(
#     [
#         LoadNIFTI(),
#         monai.transforms.AddChanneld(keys=['img', 'mask']),
#         monai.transforms.ScaleIntensityd(keys=['img', 'mask'], minv=0.0, maxv=1.0),
#         SplitMask(),
#         monai.transforms.Resized(keys=['img', 'mask'], spatial_size=(-1, 128, 128)),
#         monai.transforms.SpatialPadd(keys=['img', 'mask'], spatial_size=(16, -1, -1)),
#         monai.transforms.SpatialCropd(keys=['img', 'mask'], roi_size=(16, 128, 128), roi_center=(8, 64, 64)),
#         monai.transforms.ScaleIntensityd(keys=['mask'], minv=0.0, maxv=1.0)
#     ]
# )

compose_transform = monai.transforms.Compose(
    [
        LoadNIFTI(),
        monai.transforms.AddChanneld(keys=['img']),
        monai.transforms.ScaleIntensityd(keys=['img'], minv=0.0, maxv=1.0),
        ScaleDims(),
        FindCenter(),
    ]
)
        

In [ ]:
train_dict_list = [x for x in images]
dataset = monai.data.CacheDataset(train_dict_list, transform=compose_transform)


Loading dataset: 100%|██████████| 100/100 [00:09<00:00, 10.98it/s]


In [ ]:
data_loader = monai.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
# model = monai.networks.nets.UNETR(in_channels=1, out_channels=3, img_size=(16,128,128), feature_size=32, norm_name='batch').to(device)
model = monai.networks.nets.Unet(
    spatial_dims=3,
    in_channels=1,
    out_channels=3,
    channels = (8, 16, 32, 64),
    strides=(1, 1, 1),
    num_res_units=2,
).to(device)

model.load_state_dict(torch.load('models/trainedUNet1655812377.0684168_136.pt'))

<All keys matched successfully>

In [ ]:
# def flatten(mask):
#     out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
#     return out

# import evaluate

# for d in data_loader:
#     img = d['img']
#     label = d['mask']
#     pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
#     # print(pred.shape)
#     flatt_pred = flatten(pred[0])
#     fixed_label = 3*(label[0][0])
#     print(flatt_pred.shape)
#     print(fixed_label.shape)
    
#     print(evaluate.metrics(flatt_pred, fixed_label, [1,1,1]))
    
#     # evaluate.metrics(label, pred, [1, 1, 1])

    
# #     plt.imshow(d['img'][0][0][4], cmap='gray')
# #     plt.show()
# #     out = torch.clamp(model(d['img'].to(device)), min=0, max=1).detach().cpu().numpy()
# #     o = np.concatenate((out[0][0][4], out[0][1][4], out[0][2][4]), axis=1)
    
# #     # a = np.expand_dims(flatten(d['mask'][0, :, 4]), axis=0)
# #     b = np.expand_dims(flatten(out[0])[4],axis=0)
    
# #     # plt.imshow(a[0])
# #     # plt.show()
# #     plt.imshow(b[0])
# #     plt.show()

#     # print(d['img'].shape)
#     # print(evaluate.metrics(a, b, [1, 1, 1]))
#     # break


In [ ]:
def flatten(mask):
    out = np.where(mask[2] >=0.5 , 3, np.where(mask[1] >= 0.5, 2, np.where(mask[0]>=0.5, 1, 0)))
    return torch.Tensor(out)

for d in data_loader:
    
    img = d['img']
    # print(affine.shape)
    pred = torch.clamp(model(img.to(device)), min=0, max=1).detach().cpu().numpy()
    
    flatt_pred = flatten(pred[0])
    
    d['img'] = torch.unsqueeze(torch.unsqueeze(flatt_pred, dim=0), dim=0)
    monai.transforms.BatchInverseTransform(compose_transform, data_loader)(d)


In [ ]:
# import evaluate

# evaluate.


In [ ]:
evaluate.compute_metrics_on_files("data/training/patient057/patient057_frame01_gt.nii.gz", "results/validation_masks/patient107_ED.nii.gz")

ValueError: operands could not be broadcast together with shapes (428,512,8) (256,216,9) 

In [ ]:
image1, img_affine, img_header = load_nii("results/masks/patient029_ED.nii.gz")
image2, img_affine, img_header = load_nii("data/training/patient029/patient029_frame01_gt.nii.gz")

print(np.max(image1), np.max(image2))
print(image1.dtype)
evaluate.metrics(image1, image2, (1, 1, 1))
# print(np.max(image1), np.max(image2))


3 3
uint8


[0.9315255076770679,
 8.405,
 0.6579999999999995,
 0.8506687647521637,
 6.188,
 -0.3340000000000005,
 0.8791878172588833,
 10.527,
 -0.6160000000000014]